# Функция Python reduce(): не только функциональное программирование

Из этого туториала вы узнаете когда использовать функцию `reduce()`, как она работает и какие другие альтернативные варианты существуют.  

Статья (блокнот Jupyter) является переводом части статьи Леоданиса Позо Рамоса [Python's reduce(): From Functional to Pythonic Style](https://realpython.com/python-reduce-function/). Оригинальный текст изобилует повторами, поэтому в переводе некоторые места опущены.

В Python функция `reduce()` реализует технику [свёртки списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0). Итерируемая последовательность сворачивается до единственного значения. Функция `reduce()` популярна у разработчиков с опытом функционального программирования, однако Python может предложить кое-что ещё.

Мы изучим, как работает функция `reduce()` и как эффективно её применять. Мы также обсудим альтернативные инструменты Python, которые в определенных случаях могут быть более элегантны, удобочитаемы и эффективны.

# Функциональное программирование в Python

[Функциональное программирование]((https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5)) – это парадигма программирования, основанная на разбиении проблемы на набор отдельных функций. В идеале каждая функция принимает набор входных аргументов и возвращает выходные данные.

В функциональном программировании функции не имеют внутреннего состояния, которое может повлиять на вывод. Всякий раз при вызове функции с одинаковым набором входных аргументов вы получите один и тот же результат. Функциональное программирование избегает [изменямых типов данных](https://ru.wikipedia.org/wiki/%D0%98%D0%B7%D0%BC%D0%B5%D0%BD%D1%8F%D0%B5%D0%BC%D1%8B%D0%B9_%D1%82%D0%B8%D0%BF) и изменений состояний – работа происходит с данными, передаваемыми между функциями, а не глобальными переменными.

Ещё несколько характерных черт:
* Использование [рекурсии](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%81%D0%B8%D1%8F) вместо списков и других структур, управляющих потоком данных
* Акцент на обработке массивов
* Акцент на том, *что* должно быть вычислено, а не на том, *как* это вычислить
* Использование [чистых функций](https://ru.wikipedia.org/wiki/%D0%A7%D0%B8%D1%81%D1%82%D0%BE%D1%82%D0%B0_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8) без побочных эффектов
* Использование [функций высшего порядка](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%B2%D1%8B%D1%81%D1%88%D0%B5%D0%B3%D0%BE_%D0%BF%D0%BE%D1%80%D1%8F%D0%B4%D0%BA%D0%B0)

Python – язык, поддерживающий различные парадигмы программирования, так что он предоставляет инструменты и для функционального стиля программирования:
* Функции – [объекты первого класса](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%8A%D0%B5%D0%BA%D1%82_%D0%BF%D0%B5%D1%80%D0%B2%D0%BE%D0%B3%D0%BE_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B0)
* Возможность рекурсии
* Анонимные lambda-функции 
* Итераторы и генераторы
* Модули стандартной библиотеки: `functools` и `itertools`
* Функции [map()](https://docs.python.org/3/library/functions.html#map), [filter()](https://docs.python.org/3/library/functions.html#filter), [reduce()](https://docs.python.org/3/library/functools.html#functools.reduce), [sum()](https://docs.python.org/3/library/functions.html#sum), [len()](https://docs.python.org/3/library/functions.html#len), [any()](https://docs.python.org/3/library/functions.html#any)), [all()](https://docs.python.org/3/library/functions.html#all), [min()](https://docs.python.org/3/library/functions.html#min), [max()](https://docs.python.org/3/library/functions.html#max) и т. д.

Несмотря на то, что функциональные языки [не оказали существенного влияния](http://python-history.blogspot.com/2009/04/origins-of-pythons-functional-features.html) на Python, ещё в 1993 году существовал явный спрос на перечисленные выше подходы функционального программирования. Это привело к появлению в языке нескольких  инструментов. Согласно Гвидо ван Россуму, они [были внесены](http://www.artima.com/weblogs/viewpost.jsp?thread=98196) участником сообщества:

«Python приобрел `lambda`, `reduсe()`, `filter()` и `map()`, благодаря Lisp-хакеру, который не найдя таковых в языке, представил рабочие решения»

Спустя годы такие новые возможности, как выражения-генераторы и генераторы списков, встроенные функции `sum()`, `min()`, `max()`, `all()` и `any()`, рассматривались как более элегантная замена для `map()`, `filter()` и  `reduce()`. Так что Гвидо [планировал](http://www.artima.com/weblogs/viewpost.jsp?thread=98196) удалить их и даже `lambda` в Python 3. Но, к счастью, сообщество Python не захотело отказываться от таких популярных функций.

# Начало работы с Python reduce()

Итак `reduce()` «схлопывает» последовательность значений до единственного значения. Функция работает с любыми итерируемыми объектами – не только списками – и выполняет следующие шаги:

1. Функция или другой вызываемый объект применяется к первым двум элементам перебираемого объекта и генерируется промежуточный результат.
2. Промежуточный результат вместе с третьим элементом итерируемого объекта используются, чтобы сгенерировать следующий промежуточный результат.
3. Процесс повторяется до тех пор, пока перебираемая последовательность не будет исчерпана и не будет возвращено кумулятивное значение.

Идея, лежащая в основе `reduce()`, заключается в том, чтобы применить существующую функцию кумулятивно ко всем элементам итерируемого объекта и сгенерировать единственное конечное значение. Таким образом, функция удобна для обработки итерируемых объектов без написания явных циклов `for`. Поскольку `reduce()` написана на языке C, её внутренний цикл может быть быстрее, чем явный цикл Python.

Изначально функция `reduce()` была встроенной функцией в Python 2.x, но в третьей версии Python была перемещена в [`functools`](https://docs.python.org/3/library/functools.html#module-functools). Это решение было основано на некоторых возможных проблемах производительности и читаемости кода. На это также повлияло появление удобных специализированных функций для распространенных задач, которые ранее решались с помощью `reduce()`.

Таким образом в Python 3.x функцию необходимо импортировать из модуля `functools()`

In [1]:
from functools import reduce

Согласно [документации](https://docs.python.org/3/library/functools.html#functools.reduce), функция имеет следующую сигнатуру:

```python
functools.reduce(function, iterable[, initializer])
```

В документации Python также указано, что поведение `reduce()` примерно эквивалентно следующей функции Python:

```python
def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value
```

Функция `function` применяется к паре начальных элементов итерируемого объекта `iterable`, циклически сокращая итерируемый объект до значения `value`.

Третий необязательный аргумент функции `reduce()` – инициализатор `initializer`, предоставляющий предварительное значение для операции последовательной свёртки.

# Обязательные аргументы: `function` и `iterable`

Несмотря на то, что в официальной документации первый аргумент метода `reduce()` упоминается как «функция двух аргументов», можно передать любой вызываемый объект, принимающий два аргумента: классы и экземпляры, реализующие специальный метод `__call__()`, методы экземпляров, методы классов, статические методы и функции.

Второй обязательный аргумент – итерируемый объект: список, кортеж, объект `range`, генератор, итератор, множество, перечень значение или ключей словарей и т. д.

Чтобы понять, как работает `reduce()`, применим её для задачи нахождения суммы с выводом промежуточных результатов. Для начала нужна функция сложения двух аргументов:

In [2]:
def my_add(a, b):
    result = a + b
    print(f"{a} + {b} = {result}")
    return result

Теперь передадим в `reduce()` эту функцию и последовательность чисел:

In [3]:
numbers = [0, 1, 2, 3, 4]
reduce(my_add, numbers)

0 + 1 = 1
1 + 2 = 3
3 + 3 = 6
6 + 4 = 10


10

Всё работает, как мы описали выше: промежуточный результат используется в качестве одного из аргументов на следующей итерации вызова функции.

# Необязательный аргумент: `initializer`

При указании инициализатора, `reduce()` использует переданное значение в качестве начальной точки:

In [4]:
reduce(my_add, numbers, 100)

100 + 0 = 100
100 + 1 = 101
101 + 2 = 103
103 + 3 = 106
106 + 4 = 110


110

Момент, на который следует обратить внимание: если вы передадите значение инициализатора, `reduce()` выполнит на итерацию больше, чем без инициализатора. Если итерируемый объект окажется пустым ещё до начала свёртки, это значение будет служить значением по умолчанию. Иначе вызывается `TypeError`:

In [5]:
reduce(my_add, [], 0)

0

In [6]:
reduce(my_add, [])

TypeError: reduce() of empty sequence with no initial value

# Произведение чисел

Реализуем ту же логику (без печати промежуточных шагов) для произведения последовательности чисел:

In [7]:
numbers = [4, 5, 6, 7]
def my_prod(a, b):
    return a * b

reduce(my_prod, numbers)

840

Для совсем простых задач можно использовать анонимные lambda-функции:

In [8]:
reduce(lambda a, b: a * b, numbers)

840

Модуль `operator` содержит набор функций, соответствующий встроенным операторам Python. Для умножения – `mul()`:

In [9]:
from operator import mul
reduce(mul, numbers)

840

**Примечание**: в версии Python 3.8 в модуль `math` добавлена новая функция `prod()`, аналогичная `sum()`, но возвращающая произведение. Однако далеко не всегда есть возможность пользоваться последними версиями языка.